In [44]:
import glob 
import re
import json
chat_paths = glob.glob("./chats/*.txt")

for file_ in chat_paths:
    chats = open(file_, "r").read().split("\n")[1:]


In [35]:
def contains_deleted_or_omitted(message):
    keywords = ["omitted", "This message was deleted."]
    for keyword in keywords:
        if keyword in message:
            return True
    return False

def rm_square_brackets(message):
    return re.sub(r'\[.*?\]', '', message).strip()

In [36]:
BOT_NAME = "Pawandeep"

prev_user = ""
msg_buffer = ""
chats_clean = []
init = True

## iterate in chats
for idx, txt in enumerate(chats):
    ## remove square brackets i.e. timestamp
    txt = rm_square_brackets(txt)
    if(contains_deleted_or_omitted(txt)):
        continue

    split = txt.split(":")
    ## new line message
    if(len(split) == 1):
        msg_buffer += " "+txt
        continue

    user_name = split[0].strip()

    if init:
        init = False 
        prev_user = user_name
    
    if user_name != prev_user:
        prefix = "ASSISTANT:" if prev_user == BOT_NAME else "USER:"
        
        chats_clean.append(prefix + msg_buffer.strip())
        msg_buffer = ""
    
    prev_user = user_name
    msg_buffer += " " + split[-1].strip()

In [37]:
print(chats_clean)

['USER:Hello', 'ASSISTANT:Hi 👋 Bhai pahunch gya mein Aap btao', 'USER:Phuch gya bhai', 'ASSISTANT:👍', 'USER:🙂', 'ASSISTANT:Aur kal chloge fir?', 'USER:Haa paaji Kl jayenge Lakin main phle khatima jaunga kisi kam se Uske badd pantnagar aaunga', 'ASSISTANT:Acha 👍', 'USER:Kl ja rhe ho kya', 'ASSISTANT:Hm', 'USER:Aacha Aa rhe ho paaji', 'ASSISTANT:Aa gye', 'USER:Accha 🙂 Kb chalna hai bhai', 'ASSISTANT:Tum. Btao', 'USER:Kl hi chalte hai yaar Timing kya h ye tum decide kro 🙂', 'ASSISTANT:Chlo dekhte hain', 'USER:Train se chale ya bus se Thik h socho', 'ASSISTANT:Tum btao', 'USER:Yaar train se chalte Bus se khi traffic me fas jaye Or fhir rudrapur se fhir auto krna pdega Orr koi option ho to btao 🙂', 'ASSISTANT:Hm shi hai Mein btata hu', 'USER:👍👍', 'ASSISTANT:Yr kal to 2 classes hain bss Sorry parso', 'USER:Parso jayega ?', 'ASSISTANT:Pta nhi raat ko sochunga sote hue', 'USER:😂😂 Sochkar bta dena Aaj raat tk hii', 'ASSISTANT:👍', 'USER:Kya decide hua.. Kl hi chale firr 👍', 'ASSISTANT:Tum. Btao 

In [42]:
json_list = []

for idx in range(0, len(chats_clean), 2):
    user1, msg1 = chats_clean[idx].split(":")
    user2, msg2 = chats_clean[idx+1].split(":")
    json_list.append(
        {
            user1: msg1,
            user2: msg2
        }
    )

In [45]:
with open("./chats/clean_chats.json", "w") as f:
    f.write(json.dumps(json_list))

In [46]:
from datasets import load_dataset

In [57]:
dataset = load_dataset("./chats", data_files="clean_chats.json", split="train")

In [59]:
dataset.train_test_split(test_size=0.1)

DatasetDict({
    train: Dataset({
        features: ['ASSISTANT', 'USER'],
        num_rows: 438
    })
    test: Dataset({
        features: ['ASSISTANT', 'USER'],
        num_rows: 49
    })
})